In [ ]:
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
words = open('names.txt', 'r').read().splitlines()
words[:8]

In [ ]:
chars = sorted(list(set(''.join(words))))
stoi = {s:i+1 for i,s in enumerate(chars)}
stoi['.'] = 0
itos = {s:i for i,s in stoi.items()}
vocab_size = len(itos)

In [ ]:
# build dataset
block_size = 3 # context length: how many chars take to predict next
def build_dataset(words):
    block_size = 3
    X, Y = [], []
    for w in words:
        #print(w)
        context = [0] * block_size
        for ch in w + '.':
            ix = stoi[ch]
            X.append(context)
            Y.append(ix)
            #print(''.join(itos[i] for i in context), '--->', itos[ix])
            context = context[1:] + [ix]
    X = torch.tensor(X)
    Y = torch.tensor(Y)
    print(X.shape, Y.shape)
    return X,Y

import random
random.seed(42)
n1 = int(0.8 * len(words))
n2 = int(0.9 * len(words))
Xtr, Ytr = build_dataset(words[:n1]) # 80%
Xdev, Ydev = build_dataset(words[n1:n2]) #10%
Xte, Yte = build_dataset(words[n2:]) #10%

In [4]:
# gradients we estimate vs pytorch 
def cmp(s, dt, t):
    ex = torch.all(dt == t.grad).item()
    app = torch.allclose(dt, t.grad)
    maxdiff = (dt - t.grad).abs().max().item()
    print(f"exact: {str(ex)}, approx {str(app)}, maxdiff {maxdiff}")

In [ ]:
n_emb = 10 # dim of char vector embeddings
n_hidden = 64 # number of hidden layers 

# you want to initailize the weights with fan_in depending on tanh etc.. but now sicne they are optimizations like adam it doesn't matter much
# the idea is to keep output normal guassian , gain scales the weights due to shrinking on non-lienarity function. std = gain/sqrt(fan_in) torch kaiming normal
g = torch.Generator().manual_seed(2147483647)
C = torch.randn((vocab_size, n_emb), generator=g) #rand unfirm dist. (0,1) vs. randn gaussian dist mean 0
W1 = torch.randn((n_emb * block_size,n_hidden), generator=g) * (5/3) / (n_emb * block_size)**0.5 # gain / sqrt(fan_in)
#b1 = torch.randn(n_hidden, generator=g) * 0.1
W2 = torch.randn((n_hidden,vocab_size), generator=g) * 0.1
b2 = torch.randn(vocab_size, generator=g) * 0.1

# batch normalization
# note: initalizing with all zeros could mask an incorrect gradient, non-standard initialization
# these are usually set to 1 for the former 0 for the latter
bngain = torch.randn((1, n_hidden))*0.1 + 1.0
bnbias = torch.randn((1, n_hidden)) *0.1
#for inference

bnmean_running = torch.zeros((1, n_hidden))
bnstd_running = torch.ones((1, n_hidden))

params = [C, W1, W2, b2, bngain, bnbias]
for p in params:
    p.requires_grad = True
sum(p.nelement() for p in params)

In [ ]:
batch_size = 32
n = batch_size
ix = torch.randint(0, Xtr.shape[0], (batch_size,)) # batch_size random row index'
Xb, Yb = Xtr[ix], Ytr[ix]

In [ ]:
emb = C[Xb]
embcat = emb.view(emb.shape[0], -1)
# linear layer 1
hprebn = embcat @ W1 + b1
# batch norm layer
bnmeani = 1/n*hprebn.sum(0, keepdim=True)
bndiff = hprebn - bnmeani
bndiff2 = bndiff**2
bnvar = 1/(n-1) * (bndiff2).sum(0, keepdim=True) 
bnvar_inv = (bnvar + 1e-5)**-0.5
bnraw = bndiff * bnvar_inv
hpreact = bngain * bnraw + bnbias
# non linearity
h = torch.tanh(hpreact)
# linear layer 2
logits = h @ W2 + b2
# CE loss 
logit_maxes = logits.max(1, keepdim=True).values
norm_logits = logits = logit_maxes # num stab
counts = norm_logits.exp()
counts_sum = counts.sum(1, keepdims=True)
counts_sum_inv = counts_sum**-1
probs = counts * counts_sum_inv
logprobs = probs.log()
loss = -logprobs[range(n), Yb].mean()

for p in parameters:
    p.grad = None
for t in [logprobs, probs, counts, counts_sum, counts_sum_inv, norm_logits, logit_maxes, logits, h, hpreact,
          bnraw, bnvar_inv, bnvar, bndiff2, bndiff, phrepbn, bnmeani, em
         ]

In [ ]:
max_steps = 200000

lossi = []
for i in range(max_steps):
    # forward pass
    # minibatch (approx. gradient)
    
    
    emb = C[Xb] #emb chars to vector
    preac = emb.view(emb.shape[0], n_emb * block_size) @ W1  # + b1 - no need for b1 as it gets eaten up we have bnbias to take care of it

    # batch normalization
    bnmean = preac.mean(0, keepdim=True)
    bnstd = preac.std(0, keepdim=True)
    preac = bngain * (preac - bnmean) / bnstd+0.001 + bnbias # small num to prevent div by 0
    mom = 0.999
    with torch.no_grad():
        bnmean_running = mom * bnmean_running + (1-mom)*bnmean
        bnstd_running = mom * bnstd_running + (1-mom)*bnstd
    
    h = torch.tanh(preac) # hidden layer, tanh dead neuron 1  / -1 vanishing gradient, we can have dead neurons
    logits = h @ W2 + b2
    loss = F.cross_entropy(logits, Yb)
    
    # backward pass
    for p in params:
        p.grad = None
    loss.backward()
    
    # update
    for p in params:
        lr = 0.1 if i < 100000 else 0.01 # can be found w graph of loss against lr, step learning rate decay
        p.data += -lr * p.grad # how determine learning rate?

    if i%10000 == 0: 
        print(f'{i:7d}/{max_steps:7d}: {loss.item():.4f}')
    lossi.append(loss.log10().item())